<a href="https://colab.research.google.com/github/brayanricardo13/Analisis-Econometrico/blob/main/VAR_Value_at_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

In [2]:
pip install PyPortfolioOpt

      Successfully uninstalled cvxpy-1.0.31


# **VAR - Value at risk**

In [3]:
import yfinance as yf ,numpy as np, datetime as dt, matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
from scipy.stats import norm

In [29]:
tickers = ["TQQQ","PYPL","AMZN","MCO","QQQ","META","GOOGL","DIS","MSFT","JPM","NVDA","SSO"]
weight = np.array([11.4,6.0,11.1,0.8,17.4,16.8,10.8,5.8,2.1,2.5,0.9,2.9])

In [ ]:
Start = dt.datetime(2019,1,1)
end = dt.datetime.now()

df = yf.download(tickers,Start,end,interval="1d")["Adj Close"]
df=df.dropna()
df

In [ ]:
df.describe()

In [ ]:
df.plot()
plt.show()

In [ ]:
retourns = df.pct_change() # Retornos 
retourns.plot()
retourns

In [ ]:
cov_matrix = retourns.cov() # Matriz de varianzas y Covarianzas de los retornos
cov_matrix 

In [ ]:
corr = retourns.corr() # correlaciones 
corr

In [ ]:
retourns=retourns.corr(method="spearman") # Correlaciones Dolar 
sns.heatmap(retourns,annot=True) # Diagrama de correlacion 

In [ ]:
avg_retorns = retourns.mean() # Promedio de retornos 
avg_retorns

In [ ]:
count = retourns.count() # Numero de retornos 
count

In [30]:
Port_mean = avg_retorns @ weight
Port_std = np.sqrt(weight.T @cov_matrix @weight  )

In [31]:
Port_mean # Media del Portafolio 

26.394375412481708

In [32]:
Port_std # Desviacion estandar del portafolio 

27.140664191110403

In [ ]:
x = np.arange(-0.05,0.05,0.001)
norm_dist = norm.pdf(x,Port_mean,Port_std)
plt.plot(x,norm_dist,color="r")
plt.show()

In [34]:
confidence_level = 0.05
VaR = norm.ppf(confidence_level,Port_mean,Port_std)
VaR

-18.2480445201382

In [35]:
num_days = 5 
VaR*np.sqrt(num_days)

-40.80386800347155

# **Portafolio Eficiente**

In [ ]:
acoes = ["TQQQ","PYPL","AMZN","MCO","QQQ","META","GOOGL","DIS","MSFT","JPM","NVDA","SSO","SPXL"]
carteira = pd.DataFrame()
for i in acoes:
  carteira[i] = yf.download(i,Start,end,interval="1d")['Adj Close']
carteira=carteira.dropna()
carteira

In [ ]:
sns.set()
carteira_normalizada = carteira/carteira.iloc[0]
carteira_normalizada.plot(figsize=(21,9))

In [ ]:
retorno = carteira.pct_change()
retorno

In [ ]:
retorno_anual = retorno.mean() * 365
retorno_anual

In [ ]:
cov = retorno.cov() * 365
cov

In [ ]:
corr = retorno.corr()
corr

In [60]:
num_acoes = len(acoes)
num_carteiras = 10000

lista_retornos = []
lista_volatilidade = []
lista_pesos = []
lista_sharpe_ratio = []

In [61]:
for carteira in range(num_carteiras):
  peso = np.random.random(num_acoes)
  peso /= np.sum(peso)
  lista_pesos.append(peso)
  retorno_esperado = np.dot(peso, retorno_anual)
  lista_retornos.append(retorno_esperado)
  volatilidade = np.sqrt(np.dot(peso.T, np.dot(cov, peso)))
  lista_volatilidade.append(volatilidade)
  sharpe_ratio = retorno_esperado/volatilidade
  lista_sharpe_ratio.append(sharpe_ratio)

In [ ]:
dic_carteiras = {'Retorno': lista_retornos, 'Volatilidade': lista_volatilidade, 'Sharpe Ratio': lista_sharpe_ratio}
for contar,acao in enumerate(acoes):
  dic_carteiras[acao+' Peso'] = [Peso[contar] for Peso in lista_pesos]
portfolios = pd.DataFrame(dic_carteiras)
colunas = ['Retorno', 'Volatilidade', 'Sharpe Ratio'] + [acao+' Peso' for acao in acoes]

portfolios = portfolios[colunas]
portfolios

In [ ]:
plt.style.use('seaborn-dark')
portfolios.plot.scatter(x='Volatilidade', y='Retorno', c='Sharpe Ratio', edgecolors='black', figsize=(21,9), grid=True)
plt.xlabel('Volatilidade', fontsize=16)
plt.ylabel('Retorno Esperado', fontsize=16)
plt.title('Fronteira Eficiente de Markowitz', fontsize=22)
plt.savefig('Fronteira.png', dpi = 500)

In [ ]:
sns.scatterplot(x = 'Volatilidade', y = 'Retorno', hue = 'Sharpe Ratio', data = portfolios, legend=None)
sns.set_xticklabels('Volatilidade', fontsize = 16)
sns.set_yticklabels('Retorno Esperado', fontsize = 16)
sns.set_title('Fronteira Eficiente de Markowitz', fontsize = 22)

In [65]:
menor_volatilidade = portfolios['Volatilidade'].min()
carteira_min_variancia = portfolios.loc[portfolios['Volatilidade'] == menor_volatilidade]
carteira_min_variancia.T*100

,8008
Retorno,22.502959
Volatilidade,33.764236
Sharpe Ratio,66.647321
TQQQ Peso,0.522628
PYPL Peso,13.657357
AMZN Peso,15.303782
MCO Peso,6.961951
QQQ Peso,18.433762
META Peso,8.350411
GOOGL Peso,12.693651


In [67]:
maior_sharpe = portfolios['Sharpe Ratio'].max()
carteira_maior_sharpe = portfolios.loc[portfolios['Sharpe Ratio'] == maior_sharpe]
carteira_maior_sharpe.T*100

,3339
Retorno,44.679960
Volatilidade,40.187075
Sharpe Ratio,111.179927
TQQQ Peso,3.435076
PYPL Peso,3.545163
AMZN Peso,3.787258
MCO Peso,13.433571
QQQ Peso,8.723760
META Peso,0.279281
GOOGL Peso,11.964235
